In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import seaborn as sns

# Load data and preliminary plots

In [26]:
data = pd.read_csv('../../02_dataset/merged_df.csv')
data.drop(columns='Unnamed: 0', inplace=True)
data.head(65)

,ISO3_code,Year,pt_gdp_agriculture,oil_rent,democracy_polity,gdp,country,ethnic_fractionation_index,mean_temp,yearly_avg_rainfall,rainfall_var_t,rainfall_var_t_1,gdp_g,rainfall_diff_from_mean_norm,n_below_rainfall,oil_prod,agricultural,low_oil_high_agr,cum_rainfall_diff,mean_temp_t,mean_temp_t_1,gdp_percap,gdp_percap_pchange,gdp_pchange,unemp_rate,unemp_rate_pchange,gini,gini_pchange,population,participants,number_events,yprotest,protest_fraction
0,AFG,1960,38.627890,0.005149,-10.000000,5.621148e+09,Afghanistan,0.586,11.9,318.565807,-0.075064,-0.075064,0.000000,0.098045,0.0,False,True,True,9.804543e-02,0.033613,0.033613,1454.1108,NaN,NaN,7.908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,1961,38.627890,0.005149,-10.000000,5.621148e+09,Afghanistan,0.588,12.3,294.653108,-0.075064,-0.075064,0.000000,0.015622,0.0,False,True,True,1.136676e-01,0.033613,-0.024390,1454.1108,0.000000,0.000000,7.908,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,1962,38.627890,0.005149,-10.000000,5.621148e+09,Afghanistan,0.591,12.0,243.503638,-0.173592,-0.075064,0.000000,-0.160682,1.0,False,True,True,-4.701426e-02,-0.024390,0.033613,1454.1108,0.000000,0.000000,7.908,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,1963,38.627890,0.005149,-10.000000,5.621148e+09,Afghanistan,0.593,12.8,307.702050,0.263645,-0.173592,0.000000,0.060600,0.0,False,True,True,1.358554e-02,0.066667,-0.024390,1454.1108,0.000000,0.000000,7.908,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,1964,38.627890,0.005149,-7.000000,5.621148e+09,Afghanistan,0.596,11.3,300.362434,-0.023853,0.263645,0.000000,0.035301,0.0,False,True,True,4.888685e-02,-0.117188,0.066667,1454.1108,0.000000,0.000000,7.908,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,AFG,2020,29.975582,0.005178,-6.336735,2.062196e+10,Afghanistan,0.763,12.9,336.491733,-0.008266,0.405134,-0.023511,0.159833,0.0,False,True,True,4.908964e-01,-0.037313,-0.028986,2769.6858,-5.382508,-2.351103,11.710,4.330007,NaN,NaN,39068979.0,0.0,0.0,0.0,0.0
61,AFG,2021,33.597620,0.017870,-6.336735,1.634520e+10,Afghanistan,0.763,14.0,200.875000,-0.403031,-0.008266,-0.207388,-0.307616,1.0,False,True,True,1.832804e-01,0.085271,-0.037313,2144.1665,-22.584486,-20.738837,11.934,1.912895,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,AFG,2022,33.701430,0.005149,-6.336735,1.532523e+10,Afghanistan,0.763,14.2,268.647487,0.337386,-0.403031,-0.062402,-0.074015,2.0,False,True,True,1.092654e-01,0.014286,0.085271,1981.7102,-7.576664,-6.240173,14.100,18.149824,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,AFG,2023,34.520893,0.005149,-6.336735,1.574068e+10,Afghanistan,0.763,14.3,258.420635,-0.038068,0.337386,0.027109,-0.109265,3.0,False,True,True,-3.427814e-15,0.007042,0.014286,1992.4244,0.540654,2.710882,14.386,2.028369,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.columns

Index(['ISO3_code', 'Year', 'pt_gdp_agriculture', 'oil_rent',
       'democracy_polity', 'gdp', 'country', 'ethnic_fractionation_index',
       'mean_temp', 'yearly_avg_rainfall', 'rainfall_var_t',
       'rainfall_var_t_1', 'gdp_g', 'rainfall_diff_from_mean_norm',
       'n_below_rainfall', 'oil_prod', 'agricultural', 'low_oil_high_agr',
       'cum_rainfall_diff', 'mean_temp_t', 'mean_temp_t_1', 'gdp_percap',
       'gdp_percap_pchange', 'gdp_pchange', 'unemp_rate', 'unemp_rate_pchange',
       'gini', 'gini_pchange', 'population', 'participants', 'number_events',
       'yprotest', 'protest_fraction'],
      dtype='object')

In [6]:
feats = ['Year', 'yearly_avg_rainfall', 'rainfall_var_t', 'rainfall_var_t_1', 'mean_temp', 'pt_gdp_agriculture', 'oil_rent', 'democracy_polity', 'gdp', 'gini', 'protest_fraction']

In [14]:
feats = ['Year', 'rainfall_var_t', 'rainfall_var_t_1']

In [15]:
subsaharan = ['AGO', 'BEN', 'BWA', 'CIV', 'CAF', 'CMR', 'COD', 'COG', 'CPV', 'DJI', 'ERI', 'ETH', 'GAB', 'GHA', 'GIN', 'GMB', 'KEN', 'LSO', 'LBR', 'MDG', 'MLI', 'MRT', 'MUS', 'MWI', 'NAM', 'NER', 'NGA', 'RWA', 'SEN', 'SDN', 'SLE', 'SOM', 'SSD', 'SWZ', 'SYC', 'TCD', 'TGO', 'TZA', 'UGA', 'ZMB', 'ZWE']
sub_data = data[data['ISO3_code'].isin(subsaharan)]
print(sub_data['ISO3_code'].unique())
print(len(sub_data['ISO3_code'].unique()))

['AGO' 'BEN' 'BWA' 'CAF' 'TCD' 'COG' 'CIV' 'COD' 'DJI' 'ERI' 'ETH' 'GAB'
 'GMB' 'GHA' 'GIN' 'KEN' 'LSO' 'LBR' 'MDG' 'MWI' 'MLI' 'MRT' 'NAM' 'NER'
 'NGA' 'RWA' 'SEN' 'SLE' 'SOM' 'SDN' 'TZA' 'TGO' 'UGA' 'ZMB' 'ZWE']
35


## Very simple linear regression for sub-saharan African countries

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import root_mean_squared_error, r2_score

In Miguel et al., they use the following linear model:

$$
\mathrm{growth}_{it}=a_{i}+X_{it}b+c_{0}\Delta R_{it} + c_{1}\Delta R_{i,t-1} + d_{i}\mathrm{year}_t + e_{it},
$$
where $i$ indexes the country, $t$ indexes the year, $X_{it}$ denote the country-specific variables and $e_{it}$ is a disturbance term allowed to be correlated across years for the same country.

For now let's consider the following simpler version (which they also fit):

$$
\mathrm{growth}_{it} = a_{i} +  c_0 \Delta R_{it} + c_1 \Delta R_{i,t-1} + d_i\mathrm{year}_t.
$$

This equation fits multiple country data together, but allows for different $a_{i}$ and $d_{i}$ parameters for each country. We can construct the same fit by defining categorical variables for each one of the $n$ countries, $\delta_i$, and introducing interaction terms with the year variable:

$$
\mathrm{growth}_{it} = \beta_0  + c_0 \Delta R_{it} + c_1 \Delta R_{i,t-1} + \beta_1 \delta_{1} + \cdots + \beta_n \delta_n + \beta_{n+1} \mathrm{year} \delta_1 + \cdots + \beta_{2n} \mathrm{year} \delta_n
$$
such that when $i=1$, we get

$$
\mathrm{growth}_{1t} = \beta_0 + c_0 \Delta R_{it} + c_1 \Delta R_{i,t-1} + \beta_1  + \beta_{n+1} \mathrm{year}. 
$$

In [17]:
sub_data.dropna(inplace=True)
cols = []
for code in sub_data['ISO3_code'].unique():
    sub_data[code] = pd.get_dummies(sub_data['ISO3_code'])[code]*1
    sub_data[f'{code}_year'] = sub_data[code] * sub_data['Year']
    cols.append(code)
    cols.append(f'{code}_year')
sub_data.columns

/tmp/ipykernel_16351/1041852720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_data.dropna(inplace=True)
/tmp/ipykernel_16351/1041852720.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_data[code] = pd.get_dummies(sub_data['ISO3_code'])[code]*1
/tmp/ipykernel_16351/1041852720.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

Index(['ISO3_code', 'Year', 'pt_gdp_agriculture', 'oil_rent',
       'democracy_polity', 'gdp', 'country', 'ethnic_fractionation_index',
       'mean_temp', 'yearly_avg_rainfall', 'rainfall_var_t',
       'rainfall_var_t_1', 'gdp_g', 'rainfall_diff_from_mean_norm',
       'n_below_rainfall', 'oil_prod', 'agricultural', 'low_oil_high_agr',
       'cum_rainfall_diff', 'mean_temp_t', 'mean_temp_t_1', 'gdp_percap',
       'gdp_percap_pchange', 'gdp_pchange', 'unemp_rate', 'unemp_rate_pchange',
       'gini', 'gini_pchange', 'population', 'participants', 'number_events',
       'yprotest', 'protest_fraction', 'AGO', 'AGO_year', 'BEN', 'BEN_year',
       'BWA', 'BWA_year', 'CAF', 'CAF_year', 'TCD', 'TCD_year', 'DJI',
       'DJI_year', 'ETH', 'ETH_year', 'GAB', 'GAB_year', 'GHA', 'GHA_year',
       'GIN', 'GIN_year', 'KEN', 'KEN_year', 'LSO', 'LSO_year', 'LBR',
       'LBR_year', 'MDG', 'MDG_year', 'MWI', 'MWI_year', 'MLI', 'MLI_year',
       'MRT', 'MRT_year', 'NAM', 'NAM_year', 'NER', 'NER

In [18]:
interaction_lr = LinearRegression()
interaction_lr.fit(sub_data[feats + cols], sub_data['gdp_g'])
print(interaction_lr.score(sub_data[feats + cols], sub_data['gdp_g']))
print(root_mean_squared_error(sub_data['gdp_g'], interaction_lr.predict(sub_data[feats+cols])))

0.13481166252322507
0.059762322750470075


In the paper, they report a root mean square error of $0.07$ and $R^2=0.02$ for this fit. Let's get the coefficients and standard errors as well.

In [19]:
import statsmodels.api as sm

ols = sm.OLS(sub_data['gdp_g'], sub_data[feats+cols])
ols_result = ols.fit()
ols_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  gdp_g   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     2.224
Date:                Wed, 16 Apr 2025   Prob (F-statistic):           9.15e-07
Time:                        08:17:25   Log-Likelihood:                 1261.4
No. Observations:                 902   AIC:                            -2403.
Df Residuals:                     842   BIC:                            -2115.
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Year                 0.0006      0.000      2.823      0.005       0.000       0.001
rainfall_var_t       0.0228      0.014      1.591      0.112      -0.005       0.051
rainfall_var_t_1     0.0129      0.014      0.907      0.365      -0.015       0.041
AGO                 -0.3763      2.490     -0.151      0.880      -5.264       4.512
AGO_year            -0.0004      0.001     -0.345      0.730      -0.003       0.002
BEN                  0.2840      2.491      0.114      0.909      -4.604       5.172
BEN_year            -0.0007      0.001     -0.612      0.541      -0.003       0.002
BWA                  2.7960      2.491      1.123      0.262      -2.093       7.685
BWA_year            -0.0020      0.001     -1.642      0.101      -0.004       0.000
CAF                 -0.2060      2.490     -0.083      0.934      -5.094       4.682
CAF_year            -0.0005      0.001     -0.425      0.671      -0.003       0.002
TCD                  0.7578      2.491      0.304      0.761      -4.131       5.646
TCD_year            -0.0010      0.001     -0.804      0.421      -0.003       0.001
DJI                 -3.5560      2.491     -1.428      0.154      -8.445       1.333
DJI_year             0.0012      0.001      0.944      0.346      -0.001       0.004
ETH                 -6.2345      2.491     -2.503      0.012     -11.123      -1.346
ETH_year             0.0025      0.001      2.061      0.040       0.000       0.005
GAB                  0.1617      2.491      0.065      0.948      -4.728       5.051
GAB_year            -0.0007      0.001     -0.571      0.568      -0.003       0.002
GHA                 -1.2306      2.491     -0.494      0.621      -6.119       3.658
GHA_year          1.222e-05      0.001      0.010      0.992      -0.002       0.002
GIN                 -1.8113      2.490     -0.727      0.467      -6.699       3.077
GIN_year             0.0003      0.001      0.244      0.808      -0.002       0.003
KEN                 -1.7162      2.494     -0.688      0.492      -6.611       3.179
KEN_year             0.0002      0.001      0.201      0.841      -0.002       0.003
LSO                  4.3732      2.490      1.756      0.079      -0.515       9.261
LSO_year            -0.0028      0.001     -2.291      0.022      -0.005      -0.000
LBR                -14.3156      2.490     -5.748      0.000     -19.204      -9.427
LBR_year             0.0065      0.001      5.333      0.000       0.004       0.009
MDG                 -0.5286      2.490     -0.212      0.832      -5.417       4.360
MDG_year            -0.0004      0.001     -0.289      0.773      -0.003       0.002
MWI                 -0.4475      2.491     -0.180      0.857      -5.336       4.441
MWI_year            -0.0004      0.001     -0.315      0.753      -0.003       0.002
MLI                 -0.

The observation that yearly rainfall variation, and rainfall variation with lag is significantly different from zero doesn't hold as strongly with our data as it does with the original dataset. The model seems not to be robust to variations in the dataset (see `reproduce_res.ipynb`) caused by pulling the data from different sources.

## Try more features

In [20]:
feats = ['Year', 'yearly_avg_rainfall', 'rainfall_var_t', 'rainfall_var_t_1', 'mean_temp', 'pt_gdp_agriculture', 'oil_rent', 'democracy_polity', 'gdp', 'gini', 'protest_fraction']

In [21]:
interaction_lr = LinearRegression()
interaction_lr.fit(sub_data[feats + cols], sub_data['gdp_g'])
print(interaction_lr.score(sub_data[feats + cols], sub_data['gdp_g']))
print(root_mean_squared_error(sub_data['gdp_g'], interaction_lr.predict(sub_data[feats+cols])))

0.11477545291522262
0.06045035622993804


In [22]:
ols = sm.OLS(sub_data['gdp_g'], sub_data[feats+cols])
ols_result = ols.fit()
ols_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  gdp_g   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     5.028
Date:                Wed, 16 Apr 2025   Prob (F-statistic):           1.13e-19
Time:                        08:22:16   Log-Likelihood:                 1286.3
No. Observations:                 902   AIC:                            -2495.
Df Residuals:                     863   BIC:                            -2307.
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Year                   -0.0006      0.001     -0.759      0.448      -0.002       0.001
yearly_avg_rainfall  5.338e-05   3.54e-05      1.509      0.132   -1.61e-05       0.000
rainfall_var_t          0.0098      0.018      0.541      0.588      -0.026       0.045
rainfall_var_t_1        0.0084      0.015      0.567      0.571      -0.021       0.038
mean_temp               0.0219      0.008      2.638      0.008       0.006       0.038
pt_gdp_agriculture      0.0010      0.001      1.695      0.090      -0.000       0.002
oil_rent                0.0027      0.001      4.283      0.000       0.001       0.004
democracy_polity        0.0031      0.001      4.157      0.000       0.002       0.005
gdp                 -5.064e-14   2.96e-13     -0.171      0.864   -6.31e-13     5.3e-13
gini                   -0.0259      0.114     -0.226      0.821      -0.251       0.199
protest_fraction       -0.2292      0.253     -0.907      0.365      -0.725       0.267
AGO                    -2.8785      2.940     -0.979      0.328      -8.648       2.891
AGO_year                0.0018      0.002      0.891      0.373      -0.002       0.006
BEN                     0.1743      2.437      0.072      0.943      -4.610       4.958
BEN_year                0.0002      0.002      0.143      0.887      -0.003       0.003
BWA                     3.3282      2.415      1.378      0.169      -1.413       8.069
BWA_year               -0.0012      0.002     -0.773      0.440      -0.004       0.002
CAF                    -0.3621      2.422     -0.150      0.881      -5.115       4.391
CAF_year                0.0005      0.002      0.323      0.747      -0.003       0.004
TCD                     4.1473      2.649      1.566      0.118      -1.051       9.346
TCD_year               -0.0017      0.002     -1.011      0.312      -0.005       0.002
DJI                    -1.1812      2.447     -0.483      0.629      -5.983       3.621
DJI_year                0.0009      0.002      0.580      0.562      -0.002       0.004
ETH                    -7.9363      2.944     -2.696      0.007     -13.714      -2.159
ETH_year                0.0044      0.002      2.095      0.036       0.000       0.008
GAB                    -0.1785      2.503     -0.071      0.943      -5.091       4.734
GAB_year                0.0004      0.002      0.242      0.809      -0.003       0.004
GHA                     0.4112      2.858      0.144      0.886      -5.199       6.021
GHA_year                0.0001      0.002      0.061      0.952      -0.004       0.004
GIN                     0.2780      2.561      0.109      0.914      -4.749       5.305
GIN_year                0.0002      0.002      0.127      0.899      -0.003       0.003
KEN                     0.9921      2.762      0.359      0.720      -4.430       6.414
KEN

Adding more features does not improve the model, and does not change the conclusions about the importance of rainfall variation. Interestingly, in this case, the annual mean temperatures and the percentage of gdp from agriculture emerge as features with coefficients significantly different from zero.